In [18]:
# Tratamiento de Datos de COVID
# Fuente Datos: https://github.com/CSSEGISandData/COVID-19.git
# Practica origen Tratmaiento Información: Daniel Villanueva


# Clonar repositorio de Git
!git clone https://github.com/CSSEGISandData/COVID-19.git





fatal: destination path 'COVID-19' already exists and is not an empty directory.


In [19]:
# Listado de los ficheros que contienen Datos de COVID-19
!ls -ltr ./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports




total 9232
-rwxr-xr-x 1 root root   2785 Apr 20 21:39 02-01-2020.csv
-rwxr-xr-x 1 root root   2569 Apr 20 21:39 01-31-2020.csv
-rwxr-xr-x 1 root root   2334 Apr 20 21:39 01-30-2020.csv
-rwxr-xr-x 1 root root   2184 Apr 20 21:39 01-29-2020.csv
-rwxr-xr-x 1 root root   2102 Apr 20 21:39 01-28-2020.csv
-rwxr-xr-x 1 root root   2049 Apr 20 21:39 01-27-2020.csv
-rwxr-xr-x 1 root root   1896 Apr 20 21:39 01-26-2020.csv
-rwxr-xr-x 1 root root   1790 Apr 20 21:39 01-25-2020.csv
-rwxr-xr-x 1 root root   1695 Apr 20 21:39 01-24-2020.csv
-rwxr-xr-x 1 root root   1832 Apr 20 21:39 01-23-2020.csv
-rwxr-xr-x 1 root root   1675 Apr 20 21:39 01-22-2020.csv
-rw-r--r-- 1 root root   3551 Apr 20 21:39 02-20-2020.csv
-rw-r--r-- 1 root root   3550 Apr 20 21:39 02-19-2020.csv
-rw-r--r-- 1 root root   3588 Apr 20 21:39 02-18-2020.csv
-rw-r--r-- 1 root root   3585 Apr 20 21:39 02-17-2020.csv
-rw-r--r-- 1 root root   3510 Apr 20 21:39 02-16-2020.csv
-rw-r--r-- 1 root root   3509 Apr 20 21:39 02-15-2020.csv
-rw

Como se ve el  primer fichero y el último tienen distintos campos ( aunque tienen campos en común)



In [20]:
import numpy as np
import pandas as pd
import os
import glob # Para buscar ficheros en un sistema de archivos


# vamos a hacer lecutra del primer fichero y del ultimo fichero que hay en el directorio:
    #fichero1 = pd.read_csv('./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv')
# Hemos visto que tiene estos campos: 
# Province/State	Country/Region	Last Update	Confirmed	Deaths	Recovered
# POr lo que se van a leer de tal forma que se renombren y sean mas manejables

fichero1 = pd.read_csv('./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv').rename(columns = {'Province/State': 'Province_State',
                                                                                                                     'Country/Region': 'Country_Region',
                                                                                                                     'Last Update': 'Last_Update'
                                                                                                                     }
                                                                                                          )

fichero1.head()

# Hemos visto que el ultimo fichero tiene estos campos
# FIPS	Admin2	Province_State	Country_Region	Last_Update	Lat	Long_	Confirmed	Deaths	Recovered	Active	Combined_Key
# Le hago los mismos renames y alguno más, cómo longitud y latitud

fichero2 = pd.read_csv('./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/04-18-2020.csv').rename(columns = {'Province/State': 'Province_State',
                                                                                                                     'Country/Region': 'Country_Region',
                                                                                                                     'Last Update': 'Last_Update',
                                                                                                                     'Lat': 'Latitude',
                                                                                                                     'Long_': 'Longitude'
                                                                                                                     }
                                                                                                          )

fichero2.head()

display(fichero2.head())

# Recorre el directorio y genero una lista con todos los ficheros que cumplen el patrón

files = glob.glob('./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv')

# Ordeno la lista por los ficheros antiguos
files.sort(key =os.path.getmtime)



,FIPS,Admin2,Province_State,Country_Region,Last_Update,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-18 22:32:47,34.223334,-82.461707,15,0,0,15,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-18 22:32:47,30.295065,-92.414197,110,7,0,103,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-18 22:32:47,37.767072,-75.632346,33,0,0,33,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-18 22:32:47,43.452658,-116.241552,593,9,0,584,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-18 22:32:47,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US"


In [21]:
# Se procede a Cargar el DataFrame con todos los ficheros que tiene la lista
data = pd.DataFrame()

for file in files: # En files está la lista de ficheros que había en el directorio
  df = pd.read_csv(file).rename ( columns = {
      'Province/State': 'State',
      'Province_State': 'State',
      'Country/Region': 'Country',
      'Country_Region': 'Country',
      'Confirmed': 'ConfirmedAcumd',
      'Deaths' : 'DeathsAcum',
      'Recovered': 'RecoveredAcum',
      'Last Update': 'Last_Update',
      'Lat': 'Latitude',
      'Long_': 'Longitude'

  })
  
  df = df.assign( Date = pd.to_datetime(file[-14:-4], format = '%m-%d-%Y'), # Le metemos la tacada del fichero
                  Country = df.Country.str.strip()) # Se eliminan los blancos que puede contener el campo Country

  #Se van concatenando los dataframes a data ( que es el que se creo al comienzo vacio data = Dataframe())
  data = pd.concat((data, df), axis = 0 ) # Se concatenan los dataframes a nivel de fila ( un tipico union de bbdd)


data = data[["Date", "Admin2", "State", "Country", "ConfirmedAcumd", "DeathsAcum", "RecoveredAcum"]]

# Ahora aplicamos procedimiento de calidad del dato para estandarizar y normlizar nombres en todo el Dataset ( dataframe)
data = data.replace({
    'Bahamas, The': 'Bahamas',
    'Congo (Brazzaville)': 'Congo',
    'Congo (Kinshasa)': 'Congo',
    'Dominica': 'Dominican Republic',
    'Taiwan*': 'Taiwan',
    'Falkland Islands (Malvinas)':'Malvinas'
    })

# Se rellenan aquellos huecos de información con N/A
data = data.fillna({'State': 'N/A' , 'Admin2': 'N/A' })

# Obtenemos la información del dataframe ( dataset de datos)
data.info()

# Agrego los datos porque como estaban sucios, algún pais repetido para una fecha, pues de esta forma se obtiene una tabla más limpia
# Para agregar usamos la función de group by y hacemos un sumatorio, además hacemos que los indices sean de nuevo columnas
# porque sino al hacer un agregado los campos de agrupados pasan a ser indices

data = data.groupby(["Date", "Country", "State", "Admin2"]).agg("sum").reset_index()

data.head()

# Puedo ver los datos que me salen para españa por fechas
data[data.Country =='Spain'] #Si quiere ver una muestra añado .head(20) y saca los primeros 20

# Tal y como se presenta este dataset ahora mismo, Estados Unidos hay datos para 57 Estados, pero nosotros ahora queremos
# los datos por paises ( es decir, EEUU un único registro por día)
# Para poder hacer las operaciones hay que ordenar y luego operar
# Y además con la funcion shift(1) al estar ordenado y los datos estar totalizados ( tenemos los totales que hay cada dia)
# Para ver la evolución hay que restar al día x el día x -1 y la diferencia son los nuevos casos en las tres variables ( casos confirmados, fallecidos y recuperados)
# y esto se consigue con la funcion shift, dado que corre un registro hacia abajo, el valor de la fila 1, lo pone en la fila 2 en el campo nuevo y asi se pueden
# restar y ver la diferencia entre dias que serían los nuevos casos.

data = data.sort_values(['State', 'Country', 'Date'])

data = data.assign (
                    ConfirmedPreviuos = data.groupby(['Admin2', 'State', 'Country']).shift(1)['ConfirmedAcumd'],
                    DeathsPreviuos = data.groupby(['Admin2', 'State', 'Country']).shift(1)['DeathsAcum'],
                    RecoveredPreviuos = data.groupby(['Admin2', 'State', 'Country']).shift(1)['RecoveredAcum']
                 ).fillna({'ConfirmedPreviuos': 0, 'DeathsPreviuos': 0, 'RecoveredPreviuos':0 })


# Con los datos totatales corridos una posición ( el día 20 tiene los acumulados que estaban en el día 21 y así sucesivamente calculamos 3 nuevos cmapos)
# que seran los afectados diarios, las bajas diarias y las altas diarias

data = data.assign ( Confirmed = data.ConfirmedAcumd - data.ConfirmedPreviuos,
              Deaths = data.DeathsAcum - data.DeathsPreviuos,
             Recovered = data.RecoveredAcum - data.RecoveredPreviuos )

# Y ahora puedo borrar todas las columnas intermedias que he Usado (como son las coumnas Previus)

data.drop (['ConfirmedPreviuos','DeathsPreviuos', 'RecoveredPreviuos' ], axis = 1)
#data.head(20)

data[data.Country =='Spain'].head(20) #Si quiere ver una muestra añado .head(20) y saca los primeros 20



#Estos datos diarios los puedo agregar por pais
data_by_country = data.groupby(['Date', 'Country']).agg('sum').reset_index()
data_by_country = data_by_country.sort_values(['Country', 'Date'])
#data_by_country.info()

display(data_by_country.head(20))





<class 'pandas.core.frame.DataFrame'>
Int64Index: 95588 entries, 0 to 3071
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            95588 non-null  datetime64[ns]
 1   Admin2          95588 non-null  object        
 2   State           95588 non-null  object        
 3   Country         95588 non-null  object        
 4   ConfirmedAcumd  95569 non-null  float64       
 5   DeathsAcum      95147 non-null  float64       
 6   RecoveredAcum   95200 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 5.8+ MB


,Date,Country,ConfirmedAcumd,DeathsAcum,RecoveredAcum,ConfirmedPreviuos,DeathsPreviuos,RecoveredPreviuos,Confirmed,Deaths,Recovered
848,2020-02-24,Afghanistan,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
886,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
928,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
977,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1030,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1091,2020-02-29,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1154,2020-03-01,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1222,2020-03-02,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1297,2020-03-03,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1376,2020-03-04,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


AHORA TRAEMOS INFORMACIÓN DE OTRA FUENTE, COMO ES UNA INFORMACIÓN DE LAS POBLACIONES GLOBALES. NUMERO DE PERSONAS QUE TIENE CADA PAIS, MUJERES,  NIÑOS, ETC.

Lo trato mediante el excel "Poblac


In [26]:
# Se lee el fichero xlsx con pandas
import pandas as pd
import numpy as np

population = pd.read_excel('PoblacionMundial.xlsx', sheet_name = 'PoblacionMundial')
population.info()


# Vamos a renombrar las columnas para que tengan más sentido
population = population.rename( columns =  {
    'Country (or dependency)': 'Country',
    'Population -2020': 'Population',
    'Net Change': 'Net_Change',
    'Density (P/Km²)': 'Density',
    'Land Area (Km²)': 'Land_Area',
    'Migrants (net)': 'Migrants',
    'Fert. Rate': 'Ratio de Fertilidad',
    'Med. Age': 'Edad_Media_Pais',
    'Urban Pop %': 'Urban Pop',
    'World Share': 'Porcentaje_Representacion_Mundial'
})

display(population.head())



# Se hace un merge entre el dataframe de Covid-19 agrupado por paises ( data_by_country ) y el dataframe de población ( population)
all_data = data_by_country.merge(population, how = 'left', on = 'Country')

# Visualizamos los datos

display(all_data.head(10))

# Se podría exportar a un excel
all_data.to_excel('SalidaCovid_Poblacion.xlsx', index = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Indice                   235 non-null    float64
 1   Country (or dependency)  235 non-null    object 
 2   Population               236 non-null    object 
 3   Yearly                   236 non-null    object 
 4   Net                      236 non-null    object 
 5   Density                  236 non-null    object 
 6   Land Area                236 non-null    object 
 7   Migrants                 202 non-null    object 
 8   Fert.                    236 non-null    object 
 9   Med.                     236 non-null    object 
 10  Urban                    236 non-null    object 
 11  World                    236 non-null    object 
dtypes: float64(1), object(11)
memory usage: 22.2+ KB


,Indice,Country,Population,Yearly,Net,Density,Land Area,Migrants,Fert.,Med.,Urban,World
0,NaN,NaN,-2020,Change,Change,(P/Km²),(Km²),(net),Rate,Age,Pop %,Share
1,1.0,China,"1,439,323,776",0.39 %,"5,540,090",153,"9,388,211",-348.399,1.7,38,0.61,18.47 %
2,2.0,India,"1,380,004,385",0.99 %,"13,586,631",464,"2,973,190",-532.687,2.2,28,0.35,17.70 %
3,3.0,United States,"331,002,651",0.59 %,"1,937,734",36,"9,147,420",954.806,1.8,38,0.83,4.25 %
4,4.0,Indonesia,"273,523,615",1.07 %,"2,898,047",151,"1,811,570",-98.955,2.3,30,0.56,3.51 %


,Date,Country,ConfirmedAcumd,DeathsAcum,RecoveredAcum,ConfirmedPreviuos,DeathsPreviuos,RecoveredPreviuos,Confirmed,Deaths,Recovered,Indice,Population,Yearly,Net,Density,Land Area,Migrants,Fert.,Med.,Urban,World
0,2020-02-24,Afghanistan,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
5,2020-02-29,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
6,2020-03-01,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
7,2020-03-02,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
8,2020-03-03,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
9,2020-03-04,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,"38,928,346",2.33 %,886.592,60,652.86,-62.92,4.6,18,0.25,0.50 %
